In [14]:
import pandas as pd

df = pd.read_csv("educational_concepts.csv")
df['item'] = df['item'].str.replace("http://www.wikidata.org/entity/", "")
df['subject'] = df['subject'].str.replace("http://www.wikidata.org/entity/", "")
df = df[df["item"] != df['itemLabel']]
df = df[df["subject"] != df['subjectLabel']]
df = df.rename(columns={'item': 'conceptQID', 'itemLabel': 'conceptName', 'subject':'fieldQID', 'subjectLabel': 'fieldName' })

df_concepts = df[['conceptQID', 'conceptName']].drop_duplicates()
df_concepts.to_csv("../data/wikidata_concepts.csv", index=False)
df_fields = df[['fieldQID', 'fieldName']].drop_duplicates()
df_fields.to_csv("../data/wikidata_fields_of_study.csv", index=False)

df_relationships = df[['conceptQID', 'fieldQID']].drop_duplicates()
df_relationships.to_csv("../data/wikidata_relationships.csv", index=False)